# BERT on Repo Description

1. Construct a sentence corpus for each software type using labeled/manually validated repo descriptions
2. Calculate embeddingfor each corpus
3. Compare all repo description with each software type corpus using cosine-similarity score

Author: Cierra and Crystal

In [1]:
#bert
from sentence_transformers import SentenceTransformer, util
import torch

import pandas as pd

import re

import nltk
nltk.download("punkt")

from nltk import tokenize

[nltk_data] Downloading package punkt to /home/zz3hs/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
data = pd.read_csv('/home/zz3hs/git/dspg21oss/data/dspg21oss/oss_software_tolabel_071321.csv') #import csv
data

,slug,description,readme_text,primarylanguage,commits,prog_python_desc,prog_scala_desc,prog_rlang_desc,prog_julia_desc,prog_stat_all_desc,...,prog_lua_read,prog_web_all_read,sys_windows_read,sys_linux_read,sys_mac_read,sys_android_read,sys_virtual_read,app_blockchain_read,app_cryptocurrency_read,app_blockchain_all_read
0,fdvarela/odoo8,NaN,[![Build Status](http://runbot.odoo.com/runbot...,Python,90156.0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
1,dllsf/odootest,NaN,[![Build Status](http://runbot.odoo.com/runbot...,Python,89841.0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
2,jolevq/odoopub,Branch Público con adecuaciones para LATAM (Pa...,[![Build Status](http://runbot.odoo.com/runbot...,Python,90129.0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
3,AyoubZahid/posbox,NaN,[![Build Status](http://runbot.odoo.com/runbot...,Python,89784.0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
4,abenzbiria/clients_odoo,NaN,[![Build Status](http://runbot.odoo.com/runbot...,Python,90139.0,0,0,0,0,0,...,0,2,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1349,bazurbat/jagen,A software engineer's workspace manager and bu...,# Jagen\n\n[![Join the chat at https://gitter....,Lua,3100.0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1350,navidpati/git-github.com-substratum-substratum,NaN,![Alt text](https://i.imgur.com/HS3E6Vb.png)\n...,Java,3100.0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1351,MMAUG/BF101,NaN,BF101\n=====\n\nWarning : This is a WIP projec...,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1352,Eeylx/MaydayMaydayMayday,NaN,# MaydayMaydayMayday\nI need help! T^T\n\n---\...,NaN,NaN,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [23]:
type_name =  "app_blockchain_all_read"
data_type_i = data[data[type_name] >0][["slug","description","readme_text", type_name]]
data_type_i
cutoff_i = data_type_i[type_name].quantile(.75)
data_type_i=  data_type_i[data_type_i[type_name] >= cutoff_i]
data_type_i= data_type_i[data_type_i["description"].notnull()]
data_type_i

,slug,description,readme_text,app_blockchain_all_read
298,RomelTorres/alpha_vantage,A python wrapper for Alpha Vantage API for fin...,# alpha_vantage\n\n[![Build Status](https://tr...,2
638,SelfKeyFoundation/Identity-Wallet,Code for the SelfKey Identity Wallet,# Selfkey Identity Wallet\n\n[![Build Status](...,1
1114,SelfKeyFoundation/Identity-Wallet,Code for the SelfKey Identity Wallet,# Selfkey Identity Wallet\n\n[![Build Status](...,1


In [12]:
embedder = SentenceTransformer('paraphrase-MiniLM-L6-v2') #quicker model
#embedder = SentenceTransformer('paraphrase-mpnet-base-v2') #most accurate, long run time



In [14]:
text = "An implementation of the Grammar of Graphics in R. Statistical data visualization in Python."
corpus = tokenize.sent_tokenize(text) #sentence tokenization
print(corpus)
embeddings = embedder.encode(corpus, show_progress_bar=True) # embeddings
#embeddings

['An implementation of the Grammar of Graphics in R. Statistical data visualization in Python.']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

In [15]:
query = "matplotlib: plotting with Python"
query = "New version of SQL Fiddle based on OpenIDM"

# init a result list for scores
query_embedding = embedder.encode(query, show_progress_bar=False) 

# We use cosine-similarity and torch.topk to find the highest k scores
cos_scores = util.pytorch_cos_sim(query_embedding, embeddings)[0]

top_results = torch.topk(cos_scores, k=1)   #get the top k scores

top_results.values




tensor([0.3898])

In [ ]:
result.append(top_results.values.tolist()) #unlist the top result list
if print_result:
    print("\n\n======================\n\n")
    print("Query:", query)
    print("Results:", top_results)
    print("\nTop k=5 most similar sentences in corpus:")
    for score, idx in zip(top_results[0], top_results[1]):
        print(ai_corpus[idx], "(Score: {:.4f})".format(score))

In [3]:
def get_corpus_embeddings(dir):
    with open(dir) as f:
        ai_text = f.read()
    ai_corpus = tokenize.sent_tokenize(ai_text) #sentence tokenization
    ai_embeddings = embedder.encode(ai_corpus, show_progress_bar=True) # embeddings
    return ai_embeddings


#ai_embeddings = get_corpus_embeddings("/home/zz3hs/git/dspg21RnD/data/dspg21RnD/ai_wiki_text.txt")

FileNotFoundError: [Errno 2] No such file or directory: 'hello world lalala.'

In [ ]:
# k: number of similar sentences from AI corpus
# abstract: abstract from FEDERAL RePORTER
# print_result: if TRUE, print out the similar sentenses from AI corpus to each sentence in the abstract
def get_score(k, abstract, print_result = False):
    queries = tokenize.sent_tokenize(abstract) 

    # init a result list for scores
    result = []
    
    # Find the closest k sentences of the AI corpus for each query sentence (ML) based on cosine similarity
    top_k = min(k, len(ai_embeddings))
    
    for query in queries: #compare each sentence in the abstract to the ai corpus
        query_embedding = embedder.encode(query, show_progress_bar=False) 
        
        # We use cosine-similarity and torch.topk to find the highest k scores
        cos_scores = util.pytorch_cos_sim(query_embedding, ai_embeddings)[0]
        
        top_results = torch.topk(cos_scores, k=top_k)   #get the top k scores
        result.append(top_results.values.tolist()) #unlist the top result list
        if print_result:
            print("\n\n======================\n\n")
            print("Query:", query)
            print("Results:", top_results)
            print("\nTop k=5 most similar sentences in corpus:")
            for score, idx in zip(top_results[0], top_results[1]):
                print(ai_corpus[idx], "(Score: {:.4f})".format(score))
    return result

In [ ]:
import datetime
start_time = datetime.datetime.now()
print("Start time for calculating embeddings:", start_time)

abstracts = abstracts.assign(score= abstracts["ABSTRACT"].apply(lambda x: get_score(10,x,False)))

end_time = datetime.datetime.now()
print("Finished calculating ", len(abstracts), "of", "embedding score at", end_time)
print("It took", end_time-start_time, "to run.")
